In [1]:
#pip3 install notebook ipywidgets plotly
#jupyter nbextension enable --py widgetsnbextension
#jupyter nbextension enable --py plotlywidget

In [2]:
import igraph as ig
import pandas as pd
import webbrowser
import json
import urllib.request
import chart_studio.plotly as py
import plotly.graph_objs as go
from ipywidgets import widgets

from utils import *

import sys
sys.path.append("..")
from predict import *
from config import *

import os
parent_dir = os.path.dirname(os.getcwd())

This visualization is inspired from plotly officiel tutorials: https://plot.ly/python/v3/3d-network-graph/

In [3]:
#Load data for the graph
df_node = pd.read_csv(os.path.join(parent_dir,DF_NODE_FILENAME)).drop('Unnamed: 0',axis=1)
df_edge = pd.read_csv(os.path.join(parent_dir,DF_EDGE_FILENAME)).drop('Unnamed: 0',axis=1)

In [4]:
df_node.head()

,name,url,keywords
0,intelligent agent,https://en.wikipedia.org/wiki/Intelligent_agent,"['intelligent', 'agents', 'agent', 'autonomous..."
1,chatbot,https://en.wikipedia.org/wiki/Chatbot,"['chatbots', 'apps', 'keywords', 'conversation..."
2,artificial neural network,https://en.wikipedia.org/wiki/Artificial_neura...,"['cats', 'cat', 'neurons', 'images', 'biologic..."
3,artificial intelligence,https://en.wikipedia.org/wiki/Artificial_intel...,"['ai', 'intelligence', 'machines', 'humans', '..."
4,weak ai,https://en.wikipedia.org/wiki/Weak_AI,"['ai', 'narrow', 'weak', 'minds', 'intelligenc..."


In [5]:
df_edge.head()

,source,target
0,0,95
1,0,69
2,0,103
3,0,94
4,0,99


In [6]:
# Create placeholders for graph information
labels = df_node['name'].tolist()
color_node = ['rgba(200,200,200,0.05)' for i in range(3*len(labels))]
color_node_original = color_node.copy()
texts_to_show = [None for i in range(len(labels))]
texts_to_show_original = texts_to_show.copy()
size_node = [15 for i in range(len(labels))]
size_node_original = size_node.copy()
color_edge = ['rgba(128,128,128,0.7)' for _ in range(3*len(df_edge))]
color_edge_original = color_edge.copy()

# Create plotting items
# title, annotations, textbox_query, selector, query_answer, g = create_plot_items(df_node,df_edge,labels,color_node,texts_to_show,color_edge)

In [7]:
# import pickle
# with open('graph,pkl','wb') as f:
#     pickle.dump(g,f)

In [8]:
# Load graph and models
title, annotations, textbox_query, selector, query_answer, g = load_graph('graph,pkl')
query_bot = load_models(parent_dir)

In [9]:
def find_index_edges(nodes_ls):
    edges_idx = []
    for source in nodes_ls:
        for target in nodes_ls:
            filtered = df_edge[((df_edge.source == source) & (df_edge.target == target) ) | ( (df_edge.source == target) & (df_edge.target == source) )]
            if len(filtered)!=0:
                edges_idx.append(filtered.index.values[0])
                
    return edges_idx

In [10]:
current_selector = selector.value

# Choose the model
def response(change):
    global current_selector
    current_selector = selector.value
    make_query(textbox_query)
    
# On a new query, compute the predictions and color the nodes accordingly
def make_query(user_query):
    global current_selector
    global color_node_original
    global texts_to_show_original
    global size_node_original
    global color_edge_original
    texts_to_show = texts_to_show_original.copy()
    color_node = color_node_original.copy()
    size_node = size_node_original.copy()
    color_edge = color_edge_original.copy()
    
    query = user_query.value
    if query != "":
        preds = query_bot.make_prediction(query, current_selector)
        #preds = {10:1,11:0.8,12:0.6,13:0.4,14:0.3}
        if preds == None:
            html_text = create_text([],df_node)
            query_answer.value = html_text
        else:
            dict_colors = compute_color(preds)
        
            html_text = create_text(list(dict_colors.keys()),df_node)
            query_answer.value = html_text

            for k,v in dict_colors.items():
                size_node[k] = 30
                color_node[k] = v
                texts_to_show[k] = labels[k]
            
            edges_idx = find_index_edges(list(preds.keys()))
            for idx in edges_idx:
                color_edge[3*idx] = 'rgb(255,0,0,1)'
            

        with g.batch_update():
            g.data[1].marker.size = size_node
            g.data[1].marker.color = color_node
            g.data[1].text = texts_to_show
            g.data[0].line.color = color_edge
            
# Open url when clicking on node
def update_point(trace, points, selector):
    if len(points.point_inds) != 0:
        url = g.data[1].customdata[points.point_inds[0]]
        webbrowser.open_new_tab(url)
    
textbox_query.on_submit(make_query)
selector.observe(response, names="value")
g.data[1].on_click(update_point)

with g.batch_update():
    g.data[1].marker.color = color_node_original.copy()
    g.data[1].marker.size = size_node_original.copy()
    g.data[1].text = texts_to_show_original.copy()
    g.data[0].line.color = color_edge_original.copy()
    g.data[0].line.width = 10

In [11]:
query_bot.node2vec.wv.vocab

{'procedural reasoning system': <gensim.models.keyedvectors.Vocab at 0x7f1860c09898>,
 'jack intelligent agents': <gensim.models.keyedvectors.Vocab at 0x7f1860b8b6a0>,
 'software agent': <gensim.models.keyedvectors.Vocab at 0x7f185e318668>,
 'distributed multi-agent reasoning system': <gensim.models.keyedvectors.Vocab at 0x7f185e3185f8>,
 'agent-based model': <gensim.models.keyedvectors.Vocab at 0x7f185e318518>,
 'internet bot': <gensim.models.keyedvectors.Vocab at 0x7f185e318550>,
 'votebots': <gensim.models.keyedvectors.Vocab at 0x7f185e318908>,
 'sparse distributed memory': <gensim.models.keyedvectors.Vocab at 0x7f185e318a58>,
 'associative memory base': <gensim.models.keyedvectors.Vocab at 0x7f185e318898>,
 'artificial neural network': <gensim.models.keyedvectors.Vocab at 0x7f185e3187b8>,
 'tensor product network': <gensim.models.keyedvectors.Vocab at 0x7f185e318438>,
 'in situ adaptive tabulation': <gensim.models.keyedvectors.Vocab at 0x7f185e3184e0>,
 'radial basis function netwo

In [12]:
container = widgets.HBox(children=[textbox_query, selector])
widgets.VBox([title, annotations, container, query_answer, g])